In [ ]:
import ipyparallel
import socket
import os
import getpass
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
import funcs
import tools
from operator import itemgetter
import adaptive
adaptive.notebook_extension()

In [ ]:
import kwant

In [ ]:
client = ipyparallel.Client(
    profile='slurm2104', 
    sshserver='iffslurm.iff.kfa-juelich.de'
)
client[:].use_cloudpickle()

ids = client.ids
hosts = client[:].apply_sync(socket.gethostname)
print(ids)

notbookcwd = os.getcwd()
clustercwd = notbookcwd.replace('Home', 'Users/' + getpass.getuser())
print(len(client[:].apply_sync(os.chdir, clustercwd)))

In [ ]:
client2 = ipyparallel.Client(
    profile='slurm2102', 
    sshserver='iffslurm.iff.kfa-juelich.de'
)
client2[:].use_cloudpickle()

ids = client2.ids
hosts = client2[:].apply_sync(socket.gethostname)
print(ids)

notbookcwd = os.getcwd()
clustercwd = notbookcwd.replace('Home', 'Users/' + getpass.getuser())
print(len(client2[:].apply_sync(os.chdir, clustercwd)))

In [ ]:
def reload():
    import importlib
    import funcs
    import tools
    import systems
    
    importlib.reload(funcs)
    importlib.reload(tools)
    importlib.reload(systems)
reload()

_ = client[:].apply_sync(reload)

In [ ]:
_ = client2[:].apply_sync(reload)

In [ ]:
plt.style.use(['default', './paper.mplstyle'])

# 3D model: $(N,\langle \sigma_z \rangle)(M_z,\mu)$

In [ ]:
params_3D = dict(
    a=[10],
    a_z=[20/30],
    T=[20],
    W=[600],
    delta=[0],
    flux=[0],
    m_z=[0, 0.15],
    mu=[-0.06, 0.16],
    m0=[0],
    m1=[0],
    D=[0], 
    h_mode=[0]
    )

params_3D = get_correct_limits_3D(params_3D)

In [ ]:
qa_modes = tools.QuickAdptive(
    funcs.phase_diagram,
    fname_prefix='data_learner_phase_',
    learner_bounds_names=['m_z', 'mu'],
    arg_picker=itemgetter('modes'),
    **params_3D
)

In [ ]:
qa_modes.runner.task.result()

In [ ]:
qa_modes.runn(
    goal=lambda learner: learner.npoints > 20,
    executor=client,
    periodic_saving_folder='./data/gap',
    interval=5*60
)

In [ ]:
qa_modes.save('./data/gap')

# 3D model: $\Delta_{ind}(M_z, \mu)$

# 3D model: $FOM(M_z,\mu)$

### cross section

In [ ]:
params_3D2 = dict(
    a=[-1],   # this will be reset by get_correct_lattice_constant
    a_z=[10], # a_z=T/a_z in the code
    T=[30],
    W=[500],
    delta=[0],
    m_z=[0, 0.1],
    mu=[0], 
    h_mode=[0],
    get_delta=[False],
    p_mode=[0],
    num_bands=[100],
    k_points=[400]  # gives an error of about 3.33 * 0.12 / 800 = 0.5 meV, max(A_0) * k_max/k_points
    )

qa_gap2 = tools.QuickAdptive(
    funcs.get_fig_of_merit,
    fname_prefix='data_learner_two_gaps_',
    learner_bounds_names=['m_z'],
    arg_picker=itemgetter('E_u'),
    **params_3D2
)

### $E_{u/l}$ and $\Delta_{ind}$ 

In [ ]:
params_3D = dict(
    a=[-80],   # this will be reset by get_correct_lattice_constant
    a_z=[-10], # a_z=T/a_z in the code
    T=[np.round(20/1.12), np.round(30/1.12), np.round(40/1.12), np.round(50/1.12), np.round(60/1.12)], #
    W=[500],
    delta=[0.001],
    m_z=[0, 0.08],#[0, 0.08],
    mu=[-100], 
    h_mode=[0],
    p_mode=[0],    
    get_delta=[True],
    num_bands=[30],# for u_!=0, 30 is not enough
    k_points=[2400],  # gives an error of about 3.33 * 0.12 / 800 = 0.5 meV, max(A_0) * k_max/k_points
    sym=[False],
    u_B=[0],
    u_T=[0]
    )

In [ ]:
a_z=[-20], # a_z=T/a_z in the code
T=[np.floor(20/1.0425), np.ceil(100/1.0425)],

In [ ]:
#a, a_z, W, T, mu, m_z, params, h_mode, p_mode = funcs.optimize_parameters(a=-80, a_z=-10, W=500, T=35.5802469135802, mu=-100, m_z=m_z, params=funcs.param_list[15], sym=False, h_mode=3, p_mode=15)

In [ ]:
#executor = client.executor()
#job = executor.submit(funcs.get_fig_of_merit, a=10.5, a_z=-10, T=35.58024691358025, W=500, delta=0.001, m_z=0.0039506172839506165, mu=-100, h_mode=3, p_mode=15, 
#                      get_delta=True, num_bands=30, k_points=1500, sym=False    )

In [ ]:
qa_gap2 = tools.QuickAdptive(
    funcs.get_fig_of_merit,
    fname_prefix='data_learner_two_gaps_',
    learner_bounds_names=['m_z'],
    arg_picker=itemgetter('E_u'),
    **params_3D
)
qa_gap2.load('./data/gap')

qa_gap2.learner.strategy = "npoints"

In [ ]:
qa_gap2.runn(
    goal=lambda learner: learner.npoints > len(params_3D['T'])*20,
    executor=client2,
    periodic_saving_folder='./data/gap',
    interval=10*60
)

In [ ]:
qa_gap2.runner.live_info()

In [ ]:
qa_gap2.save('./data/gap')

In [ ]:
j = 1

which_data = "gap_u"

x = qa_gap2.learner.learners[j].to_dataframe()['x'].values
y = qa_gap2.learner.learners[j].to_dataframe()['y'].values
minn = -0.07#085
maxx = 0.11
iis = []
output = []
for i in range(len(x)):
    m_z = x[i]
    T = y[i]
    if minn< x[i] < maxx:
        iis.append(i)
        gap_u = qa_gap2.learner.learners[j].to_dataframe()['extra_data'][i]['gap_u']
        E_u= qa_gap2.learner.learners[j].to_dataframe()['extra_data'][i]['E_u']
        gap_l = qa_gap2.learner.learners[j].to_dataframe()['extra_data'][i]['gap_l']
        E_l= qa_gap2.learner.learners[j].to_dataframe()['extra_data'][i]['E_l']
        E_gap = qa_gap2.learner.learners[j].to_dataframe()['extra_data'][i]['E_gap']        
        output.append(qa_gap2.learner.learners[j].to_dataframe()['extra_data'][i][which_data])
        print(i, m_z, output[-1])

#print(np.round(E_u,3), np.round(mu_u,3), mn_u)

In [ ]:
### plt.plot(x_old,output_old,"r-")
#plt.plot(x[iis],np.array(output1)*np.array(output2),"b-")
xx = np.array(x[iis])
plt.plot(xx[xx>.001],np.array(output)[xx>.001],"b-")
plt.plot(x[np.arange(start,end+1)],new_data,"r-")
#plt.ylim(0,1e-6)
#plt.plot(x[iis],new_data,"r-")

#plt.yscale('log')
#plt.plot([np.pi*3/250]*2,[-1,1],"b--",label="beta")
#plt.plot([np.sqrt((np.pi*3/250)**2+0.0035**2)]*2,[-1,1],"r:",label="sqrt(m_0^2+beta^2)")
#plt.legend();
#plt.ylim(0,0.025)

In [ ]:
start = 4
end = 5

x0=  qa_gap2.learner.learners[j].to_dataframe()['x'][start-1]
x1=  qa_gap2.learner.learners[j].to_dataframe()['x'][end+1]

y0=  qa_gap2.learner.learners[j].to_dataframe()['extra_data'][start-1][which_data] 
y1=  qa_gap2.learner.learners[j].to_dataframe()['extra_data'][end+1][which_data]

new_data=[]
for i in np.arange(start,end+1):
    xi = qa_gap2.learner.learners[j].to_dataframe()['x'][i]
    new_data.append(y0+(y1-y0)*(xi-x0)/(x1-x0))
    qa_gap2.learner.learners[j].to_dataframe()['extra_data'][i][which_data] = new_data[-1]

In [ ]:
i = 6

#for i in range(36):
Es = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['Es']
ks = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['ks']
nu_prop_modes = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['nu_prop_modes']
energies_prop_modes = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['energies_prop_modes']
E_u = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['E_u']
E_l = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['E_l']
E_gap = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['E_gap']
mu_u = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['mu_u']
mu_l = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['mu_l']
mu_gap = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['mu_gap']
mn_u = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['mn_u']
mn_l = qa_gap2.learner.learners[j].to_dataframe()["extra_data"][i]['mn_l']

bands = qa_gap.learner.to_dataframe()["extra_data"][i]['bands']
ks_bands = qa_gap.learner.to_dataframe()["extra_data"][i]['ks_bands']

print(x[i], E_u)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,6))
#ks = np.linspace(0,20,len(Es))
for k in np.arange(len(Es)):
#    plt.plot(ks_bands[i],bands[i],'k.')
    plt.plot(ks[k]*np.ones(len(Es[k]))*10,Es[k],'k.',ms=2,markeredgewidth=0)
    
plt.plot(np.array(nu_prop_modes),energies_prop_modes,c='b',label='Prop. modes');

plt.plot(np.ones(2)*0.1,[mu_gap-E_gap/2,mu_gap+E_gap/2],"c-",lw=5,label='$E_{gap}$')
plt.plot(np.ones(2),[mu_u-E_u/2,mu_u+E_u/2],"r-",lw=5,label='$E_u$')
plt.plot(np.ones(2),[mu_l-E_l/2,mu_l+E_l/2],"g-",lw=5,label='$E_l$')

#plt.plot([0,20],np.ones(2)*m0,"g:")
#plt.plot([0,20],np.ones(2)*-mu1,"r:")

    
d = (mu_u-mu_l)*.7
plt.xlim(0,d*200);
plt.ylim(mu_l-d, mu_u+d)

plt.xlim(0,10);
plt.ylim(-.15,.1);
#plt.ylim(.1,.12);
plt.legend(loc='best');
plt.title('M_z='+str(x[i]))
plt.grid();

In [ ]:
executor = client2.executor()

In [ ]:
job = executor.submit(funcs.get_fig_of_merit,a=-80,a_z=-20,T=30,W=500,delta=0.001,m_z=0,mu=-100,h_mode=3,p_mode=15,get_delta=True,
                      num_bands=30,kpoints=1500,sym=False,u_B=0,u_T=0)

In [ ]:
res = job.result()

In [ ]:
qa_gap2.runner.task.result()

In [ ]:
qa_gap2.runner.task.result()

## evaluate results

In [ ]:
plt.plot(qa_gap2.learner.learners[0].to_dataframe()['x'])

In [ ]:
def pplot(learner):
    plot = learner.plot(tri_alpha=0.2)
    return (plot.Image + plot.EdgePaths.I + plot).cols(2)

In [ ]:
qa_gap2.learner.plot(plotter=pplot)
#qa_gap2.learner.plot()

In [ ]:
fig, ax = plt.subplots(2,2,figsize=(10,10))
m_z_selected = 0.08
data_T1 = qa_gap2.learner.learners[0].to_dataframe()
data_T2 = qa_gap2.learner.learners[1].to_dataframe()

ax[0,0].plot(data_T1['x'],data_T1['y'], 'x')
ax[0,1].plot(data_T2['x'],data_T2['y'], 'x')
ax[0,0].set_ylim(0,0.05); ax[0,1].set_ylim(0,0.05);

for i in range(len(data_T1)):
    if abs( data_T1['x'][i] - m_z_selected) < 0.005:
        ks1 = data_T1['extra_data'][i]['ks']
        Es1 = data_T1['extra_data'][i]['Es']

for i in range(len(data_T2)):
    if abs( data_T2['x'][i] - m_z_selected) < 0.005:
        ks2 = data_T2['extra_data'][i]['ks']
        Es2 = data_T2['extra_data'][i]['Es']
        
for i in np.arange(len(Es1)):
    ax[1,0].plot(ks1[i]*np.ones(len(Es1[i])),Es1[i],'k.')
    
for i in np.arange(len(Es2)):
    ax[1,1].plot(ks2[i]*np.ones(len(Es2[i])),Es2[i],'k.')
    
ax[1,0].set_title('T='+str(params_3D2['T'][0]))
ax[1,1].set_title('T='+str(params_3D2['T'][1]))
ax[1,0].set_xlim(0,0.5);  ax[1,1].set_xlim(0,0.5);
ax[1,0].set_ylim(-.2,.2); ax[1,1].set_ylim(-.2,.2);

In [ ]:
qa_plot = qa_gap

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,4))
shift = 1.12
test_T = 50/shift
x = qa_plot.learner.to_dataframe()["x"]
y = qa_plot.learner.to_dataframe()["y"]
for i in np.arange(len(x)):
    if 0.0 < x[i] < 0.1 and test_T-30 < y[i] < test_T+30:
        mn_l = qa_plot.learner.to_dataframe()["extra_data"][i]['mn_l']
        ax.plot(  x[i], y[i]*shift , ["x","o","+"][int((mn_l+1))])# qa_plot.learner.to_dataframe()["extra_data"][i]['E_gap'] , "x" )
        ax.text(  x[i], y[i]*shift , s=str(i))# qa_plot.learner.to_dataframe()["extra_data"][i]['E_gap']*1.05  )

In [ ]:
i = 1
T = y[i]
m_z = x[i]
print(T,m_z)


Es = qa_plot.learner.to_dataframe()["extra_data"][i]['Es']
nu_prop_modes = qa_plot.learner.to_dataframe()["extra_data"][i]['nu_prop_modes']
energies_prop_modes = qa_plot.learner.to_dataframe()["extra_data"][i]['energies_prop_modes']
E_u   = qa_plot.learner.to_dataframe()["extra_data"][i]['E_u']
E_l   = qa_plot.learner.to_dataframe()["extra_data"][i]['E_l']
E3_u   = qa_plot.learner.to_dataframe()["extra_data"][i]['E3_u']
E3_l   = qa_plot.learner.to_dataframe()["extra_data"][i]['E3_l']
E_gap = qa_plot.learner.to_dataframe()["extra_data"][i]['E_gap']
mu_u  = qa_plot.learner.to_dataframe()["extra_data"][i]['mu_u']
mu_l  = qa_plot.learner.to_dataframe()["extra_data"][i]['mu_l']
mu3_u  = qa_plot.learner.to_dataframe()["extra_data"][i]['mu3_u']
mu3_l  = qa_plot.learner.to_dataframe()["extra_data"][i]['mu3_l']
mu_gap= qa_plot.learner.to_dataframe()["extra_data"][i]['mu_gap']
mn_u  = qa_plot.learner.to_dataframe()["extra_data"][i]['mn_u']
mn_l  = qa_plot.learner.to_dataframe()["extra_data"][i]['mn_l']

print(E_u, mu_u, mn_u)
print(E_l, mu_l, mn_l)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,6))
ks = np.linspace(0,20,len(Es))
for i in np.arange(len(Es)):
#    plt.plot(ks_bands[i],bands[i],'k.')
    plt.plot(ks[i]*np.ones(len(Es[i])),Es[i],'k.')
    
plt.plot(np.array(nu_prop_modes),energies_prop_modes,c='b',label='Prop. modes');

plt.plot(np.ones(2)*0.1,[mu_gap-E_gap/2,mu_gap+E_gap/2],"c-",lw=5,label='$E_{gap}$')
plt.plot(np.ones(2),[mu_u-E_u/2,mu_u+E_u/2],"r-",lw=5,label='$E_u$')
plt.plot(np.ones(2),[mu_l-E_l/2,mu_l+E_l/2],"g-",lw=5,label='$E_l$')
plt.plot(np.ones(2)*3,[mu3_u-E3_u/2,mu3_u+E3_u/2],"y-",lw=5,label='$E_{3,u}$')
plt.plot(np.ones(2)*3,[mu3_l-E3_l/2,mu3_l+E3_l/2],"b-",lw=5,label='$E_{3,l}$')

d = (mu_u-mu_l)*1.5
plt.ylim(mu_l-d, mu_u+d)
#plt.ylim(-0.0,.1);

plt.xlim(0,d*200);
plt.ylim(-.15,.15); plt.xlim(0,30);
plt.legend(loc='best');
plt.grid();

------

In [ ]:
extra_data = qa_gap.learner.to_dataframe()["extra_data"]
Es = [extra_data[i]['Es'].flatten() for i in np.arange(len(extra_data))]
ks = [np.repeat(extra_data[i]['ks'],50) for i in np.arange(len(extra_data))]

In [ ]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.plotting import figure, output_file, show

x = ks[0]
y = dict()
for i in np.arange(len(ks)):
    y[str(i)] = Es[i]    

inital_data = '0'

source_visible = ColumnDataSource(data=dict(
    x=x, y=y[inital_data]))
source_available = ColumnDataSource(data=y)
  
# Create plots and widgets
plot = figure(x_range=(0, 1), y_range=(-.05, .15))
  
plot.scatter('x', 'y', source=source_visible, line_width=1, line_alpha=0.5)
  
# Create Slider object
slider = Slider(value=0,
                start=0,
                end=0.1,
                step=0.1/51,
                title='Value of Mz')
  
# Adding callback code
callback = CustomJS( args=dict(source_visible=source_visible,
              source_available=source_available), code="""
        var selected_function = parseInt(50*cb_obj.value/0.1);
        var data_visible   = source_visible.data;
        var data_available = source_available.data;
        data_visible.y = data_available[selected_function];
        source_visible.change.emit();
    """)

slider.js_on_change('value', callback)

layout = column(slider, plot)
  
output_file('exam.html')
  
show(layout)

In [ ]:
params_3D = dict(
    a=[10],    
    a_z=[20/30],
    T=[20],
    W=[600],
    delta=[5e-3],
    flux=[0],
    m_z=[0, 0.15],
    mu=[-0.06, 0.16],
    m0=[0],
    m1=[0],
    D=[0], 
    h_mode=[0]
    )

params_3D = get_correct_limits_3D(params_3D)

In [ ]:
import tools
import funcs
from operator import itemgetter

qa_top = tools.QuickAdptive(
    funcs.gap_search_k_fast,
    fname_prefix='data_learner_gap_new_',
    learner_bounds_names=['m_z', 'mu'],
    arg_picker=itemgetter('gap'),
    **params_3D
)

qa_mn = tools.QuickAdptive(
    funcs.calculate_majorana_num,
    fname_prefix='data_learner_mn_',
    learner_bounds_names=['m_z', 'mu'],
    arg_picker=itemgetter('mn'),
    **params_3D
)

In [ ]:
qa_top.load('./data/gap')
qa_mn.load('./data/gap')

In [ ]:
qa_top.runn(
    goal=lambda learner: learner.npoints > 100,
    executor=client,
    periodic_saving_folder='./data/gap',
    interval=5*60
)

In [ ]:
qa_mn.runn(
    goal=lambda learner: learner.npoints > 100,
    executor=client,
    periodic_saving_folder='./data/gap',
    interval=5*60
)

In [ ]:
qa_top.save('./data/gap')
qa_mn.save('./data/gap')

---
---

# 2D model: $FOM(M_z,\mu)$

In [ ]:
executor = client.executor()
job = executor.submit(funcs.get_fig_of_merit, a=4, a_z=4, T=4, W=250, delta=0.001, m_z=0.01, 
                      mu=0, h_mode=1, p_mode=0, get_delta=True, num_bands=30, k_points=800, sym=False, u_B=-.1, u_T=-.1 )

In [ ]:
res = job.result()

In [ ]:
Es = res['Es']
ks = res['ks']
nu_prop_modes = res['nu_prop_modes']
energies_prop_modes = res['energies_prop_modes']
E_u = res['E_u']
E_l = res['E_l']
E_gap = res['E_gap']
mu_u = res['mu_u']
mu_l = res['mu_l']
mu_gap = res['mu_gap']
gap_u = res['gap_u']

In [ ]:
#plt.plot(res[0], res[1],"k.");
plt.plot(ks, Es, "k.");
plt.ylim(-.15,.15)

### cross section

In [ ]:
#Make cross section
params_3D = dict(
    a=[-1],   # this will be reset by get_correct_lattice_constant
    a_z=[10], # a_z=T/a_z in the code
    T=[100],
    W=[500],
    delta=[0],
    m_z=[0, 0.1],
    mu=[0], 
    h_mode=[0],
    get_delta=[False],
    p_mode=[0],
    num_bands=[100],
    k_points=[400]  # gives an error of about 3.33 * 0.12 / 800 = 0.5 meV, max(A_0) * k_max/k_points
    )

qa_gap = tools.QuickAdptive(
    funcs.get_fig_of_merit,
    fname_prefix='data_learner_two_gaps_',
    learner_bounds_names=['m_z'],
    arg_picker=itemgetter('E_u'),
    **params_3D
)

### ONLY $E_{u/l}$

In [ ]:
params_3D = dict(
    a=[10],   # this will be reset by get_correct_lattice_constant
    a_z=[10], # a_z=T/a_z in the code
    T=[4],
    W=[500],
    delta=[0],
    m_z=[0, 0.05],
    mu=[0], 
    h_mode=[1],
    p_mode=[0],    
    get_delta=[False],
    num_bands=[50],
    k_points=[800],  # gives an error of about 3.33 * 0.12 / 800 = 0.5 meV, max(A_0) * k_max/k_points
    sym=[False]
    )

### $E_{u/l}$ and $\Delta_{ind}$ 

In [ ]:
params_2D = dict(
    a=[4],   # this will be reset by get_correct_lattice_constant
    a_z=[4], # a_z=T/a_z in the code
    T=[6],
    W=[500],
    delta=[0.001],
    m_z=[0.001, 0.08],
    mu=[-100], 
    h_mode=[1],
    p_mode=[2],
    get_delta=[True],
    num_bands=[30],  # for p_mode=0,1 this has to be 
    k_points=[3000], # gives an error of about 1 * 0.3 / 1500 = 0.2 meV, hv_F * k_max/k_points
    sym=[False],
    u_B=[0],
    u_T=[0]    
    )

In [ ]:
qa_gap = tools.QuickAdptive(
    funcs.get_fig_of_merit,
    fname_prefix='data_learner_two_gaps_',
    learner_bounds_names=['m_z'],
    arg_picker=itemgetter('E_u'),
    **params_2D
)
qa_gap.load('./data/gap')

qa_gap.learner.strategy = "npoints"

In [ ]:
qa_gap.runner.task.result()

In [ ]:
qa_gap.runn(
    goal=lambda learner: learner.npoints > len(params_2D['p_mode'])*len(params_2D['T'])*20,
    executor=client2,
    periodic_saving_folder='./data/gap',
    interval=10*60
)

In [ ]:
qa_gap.runner.live_info()

In [ ]:
qa_gap.save('./data/gap')

In [ ]:
executor = client.executor()
job = executor.submit(funcs.get_fig_of_merit, a=4, a_z=4, T=2, W=500, delta=0.001, m_z=0, mu=mu1, h_mode=1, p_mode=0, get_delta=True, num_bands=30, k_points=3000, sym=False)

In [ ]:
param_pos = 0*6+(2-1)
H_params = funcs.H_eff_parameters[param_pos]
m0   = H_params['Delta']/2
m1   = H_params['B']
D    = H_params['D']
v_F  = H_params['v_F']

mu0 = -(m_z)*D/m1 #for large magnetization the interesting physics will be around the chiral edge mode
mu1 = -(m_z)*D/m1 + (D*m1)/abs(D*m1)*abs(m_z) #otherwise find the middle of the gap

In [ ]:
res = job.result()

In [ ]:
#res['E_gap']
ks, Es = res
for i in range(len(Es)):
    plt.plot(ks[i]*np.ones(len(Es[i])), Es[i],".")
    
plt.plot([0,1.2],[mu0]*2,"g--")
plt.plot([0,1.2],[mu1]*2,"r--")

In [ ]:
qa_gap.learner.learners[j].to_dataframe()['extra_data'][i]['E_gap']# = res['E_gap']

In [ ]:
j = 0

which_data = "E_u"

x = qa_gap.learner.learners[j].to_dataframe()['x'].values
y = qa_gap.learner.learners[j].to_dataframe()['y'].values
minn = -0.07#085
maxx = 0.11
iis = []
output = []
for i in range(len(x)):
    m_z = x[i]
    T = y[i]
    if minn< x[i] < maxx:
        iis.append(i)
        gap_u = qa_gap.learner.learners[j].to_dataframe()['extra_data'][i]['gap_u']
        E_u= qa_gap.learner.learners[j].to_dataframe()['extra_data'][i]['E_u']
        gap_l = qa_gap.learner.learners[j].to_dataframe()['extra_data'][i]['gap_l']
        E_l= qa_gap.learner.learners[j].to_dataframe()['extra_data'][i]['E_l']
        E_gap = qa_gap.learner.learners[j].to_dataframe()['extra_data'][i]['E_gap']        
        output.append(qa_gap.learner.learners[j].to_dataframe()['extra_data'][i][which_data])
        print(i, m_z, output[-1])

#print(np.round(E_u,3), np.round(mu_u,3), mn_u)

In [ ]:
### plt.plot(x_old,output_old,"r-")
#plt.plot(x[iis],np.array(output1)*np.array(output2),"b-")
plt.plot(x[iis],output,"b-")
#plt.ylim(0,1e-6)
#plt.plot(x[iis],new_data,"r-")

#plt.yscale('log')
#plt.plot([np.pi*3/250]*2,[-1,1],"b--",label="beta")
#plt.plot([np.sqrt((np.pi*3/250)**2+0.0035**2)]*2,[-1,1],"r:",label="sqrt(m_0^2+beta^2)")
#plt.legend();
#plt.ylim(0,0.025)

In [ ]:
xi = qa_gap.learner.learners[j].to_dataframe()['x']
new_data=np.exp(-x[iis]*500/v_F)*.00001
    #qa_gap.learner.learners[j].to_dataframe()['extra_data'][i][which_data] = 0#y0 + (y1-y0)*(xi-x0)/(x1-x0)

In [ ]:
i = 51

#for i in range(36):
Es = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['Es']
ks = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['ks']
nu_prop_modes = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['nu_prop_modes']
energies_prop_modes = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['energies_prop_modes']
E_u = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['E_u']
E_l = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['E_l']
E_gap = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['E_gap']
mu_u = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['mu_u']
mu_l = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['mu_l']
mu_gap = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['mu_gap']
mn_u = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['mn_u']
mn_l = qa_gap.learner.learners[j].to_dataframe()["extra_data"][i]['mn_l']

bands = qa_gap.learner.to_dataframe()["extra_data"][i]['bands']
ks_bands = qa_gap.learner.to_dataframe()["extra_data"][i]['ks_bands']

print(x[i], E_u)

In [ ]:
mu_u

In [ ]:
param_pos = params_2D['p_mode'][0]*6+(params_2D['T'][j]-1)
H_params = funcs.H_eff_parameters[param_pos]
m0   = H_params['Delta']/2
m1   = H_params['B']
D    = H_params['D']
v_F  = H_params['v_F']
m_z  = x[i]

#mu0 = (m0-m_z)*D/m1 #for large magnetization the interesting physics will be around the chiral edge mode
#mu1 = (m0-m_z)*D/m1 + (D*m1)/abs(D*m1)*abs(m_z) #otherwise find the middle of the gap

if abs(m_z) > abs(m0) or m0*m1 > 0:
    mu0 = (m0-m_z)*D/m1 #for large magnetization the interesting physics will be around the chiral edge mode
else:
    mu0 = 0

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,6))
#ks = np.linspace(0,20,len(Es))
for k in np.arange(len(Es)):
#    plt.plot(ks_bands[i],bands[i],'k.')
    plt.plot(ks[k]*np.ones(len(Es[k]))*10,Es[k],'k.',ms=2,markeredgewidth=0)
    
plt.plot(np.array(nu_prop_modes),energies_prop_modes,c='b',label='Prop. modes');

plt.plot(np.ones(2)*0.1,[mu_gap-E_gap/2,mu_gap+E_gap/2],"c-",lw=5,label='$E_{gap}$')
plt.plot(np.ones(2),[mu_u-E_u/2,mu_u+E_u/2],"r-",lw=5,label='$E_u$')
plt.plot(np.ones(2),[mu_l-E_l/2,mu_l+E_l/2],"g-",lw=5,label='$E_l$')

#plt.plot([0,20],np.ones(2)*m0,"g:")
#plt.plot([0,20],np.ones(2)*-mu1,"r:")

    
d = (mu_u-mu_l)*.7
plt.xlim(0,d*200);
plt.ylim(mu_l-d, mu_u+d)

plt.xlim(0,10);
plt.ylim(-.15,.1);
#plt.ylim(.1,.12);
plt.legend(loc='best');
plt.title('M_z='+str(x[i]))
plt.grid();

## 2D model: $\Delta_{ind}(M_z, \mu)$

In [ ]:
M0 =-0.005
M1 =15
D  =-14
syst_W = 500
u_B=0

In [ ]:
params_Chen = dict(mu_leads=0,
                   mu_sys=0,
                   hbar=1,
                   v_F=3,
                   m0=M0,
                   m1=M1,
                   D=D,
                   u_T=u_B,
                   u_B=u_B,
                   S_imp=0,
                 )

In [ ]:
which_param = 'Mz'
#which_param = 'W'

max_Mz = 0.045
max_mu = 0.05

if which_param == 'W':
    p_bounds = [200,1500]
else:
    p_bounds = [0,max_Mz]
#if D == 0:
mu_bounds = [-u_B/2-max_mu,-u_B/2+max_mu]#(-max_mu+(M0-Mz)*D/M1,max_mu+(M0-Mz)*D/M1)
#else:
#    mu_bounds = [-u_B/2-max_mu+0.005,-u_B/2+max_mu+0.005]#(-max_mu+(M0-Mz)*D/M1,max_mu+(M0-Mz)*D/M1)

npoints = 10000

In [ ]:
#mu_bounds = [-.1,.1]

In [ ]:
#mu_bounds = [-u_B/2-max_mu,-0.013]
#p_bounds = [0,0.005]
#mu_bounds = [-.04,.04]

In [ ]:
params_2D = dict(
    a=[4],
    a_z=[4],
    T=[1],
    W=[syst_W],
    mu=mu_bounds,
    m_z=p_bounds,
    delta=[0.001],
    h_mode=[1],
    params=[params_Chen],
    )

In [ ]:
params_app = params_2D.copy()
params_app['m_z'] = [0.04]
params_app['mu'] = [-.045,.055]
#params_app['mu'] = [.032,.034]
#params_app['mu'] = [.032,.055]

In [ ]:
executor = client.executor()
job = executor.submit(funcs.gap_search_k, a=4, a_z=4, T=1, W=syst_W, mu=0.025, m_z=.03, delta=0.001, h_mode=1, params=params_Chen )

In [ ]:
res = job.result()

In [ ]:
qa_ph = tools.QuickAdptive(
    funcs.get_modes,
    fname_prefix='data_learner_modes_',
    learner_bounds_names=['m_z', 'mu'],
    arg_picker=itemgetter('modes'),
    **params_2D
)

qa_top = tools.QuickAdptive(
    funcs.gap_search_k,
    fname_prefix='data_learner_gap_new_',
    learner_bounds_names=['m_z', 'mu'],
    arg_picker=itemgetter('gap'),
    **params_2D
)

qa_mn = tools.QuickAdptive(
    funcs.calculate_majorana_num,
    fname_prefix='data_learner_mn_',
    learner_bounds_names=['m_z', 'mu'],
    arg_picker=itemgetter('mn'),
    **params_2D
)

In [ ]:
qa_ph = tools.QuickAdptive(
    funcs.get_modes,
    fname_prefix='data_learner_modes_',
    learner_bounds_names=['mu'],
    arg_picker=itemgetter('modes'),
    **params_app
)

qa_top = tools.QuickAdptive(
    funcs.gap_search_k,
    fname_prefix='l_g_',
    learner_bounds_names=['mu'],
    arg_picker=itemgetter('gap_log'),
    **params_app
)

qa_mn = tools.QuickAdptive(
    funcs.calculate_majorana_num,
    fname_prefix='l_mn_',
    learner_bounds_names=['mu'],
    arg_picker=itemgetter('mn'),
    **params_app
)

In [ ]:
qa_ph.load('./data/gap')
qa_top.load('./data/gap')
qa_mn.load('./data/gap')

In [ ]:
#qa_ph.runn(
#    goal=lambda learner: learner.npoints > npoints,
#    executor=client,
#    periodic_saving_folder='./data/gap',
#    interval=5*60
#)

In [ ]:
npoints = 5000

In [ ]:
qa_top.runn(
    goal=lambda learner: learner.npoints > npoints,
    executor=client,
    periodic_saving_folder='./data/gap',
    interval=5*60
)

In [ ]:
qa_mn.runn(
    goal=lambda learner: learner.npoints > npoints,
    executor=client,
    periodic_saving_folder='./data/gap',
    interval=5*60
)

In [ ]:
#qa_ph.runner.live_info()
qa_top.runner.live_info()
qa_mn.runner.live_info()

## Analytical Mz

In [ ]:
from matplotlib.patches import Rectangle

In [ ]:
e_plot = qa_top.learner.learners[0].to_numpy()[:,0]
delta_plot  = np.array([i['gap'] for i in qa_top.learner.learners[0].to_dataframe()['extra_data']])/params_app['delta'][0]

e_plot_mn = qa_mn.learner.learners[0].to_numpy()[:,0]
mn_plot  = np.array([i['mn'] for i in qa_mn.learner.learners[0].to_dataframe()['extra_data']])

In [ ]:
v  = params_app['params'][0]['v_F']
MZ = params_app['m_z'][0]
W  = params_app['W'][0]
DD = -14
g  = np.sqrt((M1+DD)/(M1-DD))
alpha = DD/M1

Es = np.linspace(params_app['mu'][0],0.032,100)
ks = (Es+(M0+MZ)*DD/M1)/(-v*np.sqrt(1-DD**2/M1**2))
kappa_asym = (M0+MZ)/v*np.sqrt(1-DD**2/M1**2)-D*ks/M1-M1/v*np.sqrt(1-DD**2/M1**2)*ks**2

#delta_asym = 2*kappa_asym*W*1/(np.exp(kappa_asym*W)-np.exp(-kappa_asym*W))*g/(1+g**2)
#delta_asym = 2*kappa_asym*W*np.exp(-kappa_asym*W)*g/(1+g**2)
delta_asym = kappa_asym*W*np.exp(-kappa_asym*W)*np.sqrt(1-alpha**2)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(3.5,1.75),layout='constrained')

boundaries = [-.1]+list(np.array(e_plot_mn[:-1])[np.array([mn_plot[i]!=mn_plot[i+1] for i in range(len(mn_plot)-1)])])+[.1]
for i in np.arange(len(boundaries)-1):
    ax.add_patch(Rectangle((1e3*boundaries[i],1e-12),1e3*(boundaries[i+1]-boundaries[i]),1e2,color=["r","k"][(i-1)%2],alpha=.2,lw=0))

ax.plot(1e3*np.array(params_app['mu']),[1,1],"darkgrey",ls="-",lw=1);

ax.plot([1e3*(-D*(M0+params_app['m_z'][0])/M1)]*2,[1e-12,1e2],"g--",label="$E_\\text{DP}^+$",lw=1)
ax.plot([1e3*(M0+params_app['m_z'][0])]*2,[1e-12,1e2],"k--",lw=1,label="$\\pm|m_0+M_z|$")
ax.plot([1e3*(-M0-params_app['m_z'][0])]*2,[1e-12,1e2],"k--",lw=1)
ax.plot(1e3*e_plot,delta_plot,"r-",lw=1,label="Numerical")
#if(params_app['m_z'][0] == 0.03):
    #delta_an = delta_an_Mz30
    #ax.plot(np.linspace(-.1,.1,201)[np.linspace(-.1,.1,201)<0.02105],np.array(delta_an)[np.linspace(-.1,.1,201)<0.02105],"-",lw=2,label="Analytical")
#elif(params_app['m_z'][0] == 0.04):
    #delta_an = delta_an_Mz40
    #ax.plot(np.linspace(-.1,.1,201)[np.linspace(-.1,.1,201)<0.0318],np.array(delta_an)[np.linspace(-.1,.1,201)<0.0318],"-",lw=2,label="Analytical")
#else:
#    delta_an = delta_an_Mz80
ax.plot(1e3*Es[1/kappa_asym*2.5<W],delta_asym[1/kappa_asym*2.5<W],"b-",lw=1,label="Analytical")
ax.set_xlabel("$\\mu$ [meV]")
ax.set_ylabel("$\\Delta_{\\mathrm{ind}}/\\Delta_0$")
ax.set_xlim(1e3*np.array(params_app['mu']))
ax.set_ylim(1e-4,1e1)
ax.legend(loc='upper left',frameon=True,fontsize=8)
ax.set_yscale('log')

caption = ['(c)']
for i in range(len(caption)):
    if i == 0:
        sshift = -0.15
    else:
        sshift = -0.15
    ax.text(
        sshift, .985, 
        caption[i],
        transform=ax.transAxes)
plt.savefig("deltaind_num_v_analyt_Mz"+str(+params_app['m_z'][0])+".png",dpi=300)
plt.savefig("deltaind_num_v_analyt_Mz"+str(+params_app['m_z'][0])+".pdf",bbox_inches='tight')

In [ ]:
qa_ph.save('./data/gap')
qa_top.save('./data/gap')
qa_mn.save('./data/gap')

---

In [ ]:
#executor = client.executor()

In [ ]:
#import funcs

In [ ]:
#job = executor.submit(funcs.sigma_z_expec_value,10,20/30,20,200,0,0,0.1,0,0,0,0,0)

In [ ]:
#Res = job.result()

In [ ]:
#Res

In [ ]:
E_k0 = E_k0.real
    wfs_k0 = wfs_k0[:, E_k0 > 0]
    E_k0 = E_k0[E_k0 > 0]
    rhos = get_rhos(lead, sum=True)
    rhos_p_k0 = np.array([rhos['p'](wf) for wf in wfs_k0.T])
    rhos_h_k0 = np.array([rhos['h'](wf) for wf in wfs_k0.T])
    densitys_ph = rhos_p_k0 - rhos_h_k0
    fist_h_band_index = np.argmax(densitys_ph > 0)
    fist_h_band_E = E_k0[fist_h_band_index]#Res[0]

In [ ]:
b1, b2, g = funcs.gap_zero_k(10,30,48,200,0,0,-0.12,0,0,0,0,0)

In [ ]:
b = funcs.gap_zero_k(10,30,50,200,0,0,0.15,0,0,0,0,0)

In [ ]:
data['x']import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#plt.plot(np.zeros(len(b1)),b1,"X")
#plt.plot(np.ones(len(b2)),b2,"X")
#plt.ylim(-0.04,-0.0375)

In [ ]:
#b1[21], b1[20],b1[19],b1[18],b1[17]

In [ ]:
#b2[21],b2[20],b2[19],b2[18],b2[17]

In [ ]:
qa_gap.runner.task.result()

---

In [ ]:
import holoviews as hvtop
from holoviews import opts

def get_plotter(xlabel='x', ylabel='y', clabel='G (e²/h)', clim=(0,0.005)):
    def plotter(learner):    
        plot = learner.plot(tri_alpha=0.2)
        plot = plot.options(
            opts.Image(
                colorbar=True,
                frame_height=400,
                frame_width=400,
                xlabel=xlabel,
                clim=clim,
                ylabel=ylabel,
                clabel=clabel
            )
        )
        return plot
    return plotter

In [ ]:
from scipy.interpolate.interpnd import LinearNDInterpolator

## Plotters

In [ ]:
import matplotlib.patches as mpatches
def create_topological_simple(learner_inv,learner_gap,params,Max,replot):
    
    if not replot:    
        gap_interploated=learner_gap.interpolated_on_grid(n=1000)
        inv_interploated=learner_inv.interpolated_on_grid(n=1000)

        top = inv_interploated[2] > 0
        not_top = inv_interploated[2] < 0
        x, y, z = gap_interploated[0], gap_interploated[1], gap_interploated[2]
        
        z_topological = z.copy()
        z_topological[top] = -1
        
        z_not_topological = z.copy()
        z_not_topological[not_top] = -1

    else:
        x = np.linspace(*p_bounds,1000)        
        y = np.linspace(*mu_bounds,1000)
        
        z_topological_one = z_topological_one_replot
        z_topological_odd = z_topological_odd_replot
        z_not_topological_even = z_not_topological_even_replot   

    fig, ax = plt.subplots(1,1,figsize=(6,6),layout='constrained')

    im1 = ax.contourf(x,y,z_topological.T,vmin=0,vmax=Max,levels=np.append(np.linspace(0,Max,50),1),cmap="Reds")#,params_Chen,0.0025)    
    im3 = ax.contourf(x,y,z_not_topological.T,vmin=0,vmax=Max,levels=np.append(np.linspace(0,Max,50),1),cmap="gray_r")#,params_Chen,0.0025)

    cbar = fig.colorbar(im1, ax=ax,ticks=np.linspace(0,Max,5))
    Max_Delta = np.round(Max/params["delta"][0],1)
    cbar.set_ticklabels( [str(np.round(n,2))+"$\\Delta$" for n in np.linspace(0,Max_Delta,5)])

    ax.set_xlabel("$M_z$ [meV]");
    ax.set_ylabel("$\mu$ [meV]");
    xs = np.linspace(p_bounds[0],.01*(p_bounds[1]//.01),int((p_bounds[1]-p_bounds[0])/.01)+1)
    ax.set_xticks(xs);
    ys = np.linspace(-.02-(mu_bounds[1]-mu_bounds[0])/2,.02+(mu_bounds[1]-mu_bounds[0])/2,int((.04+mu_bounds[1]-mu_bounds[0])/.01)+1)
    ax.set_yticks(ys);
    ax.set_ylim(*mu_bounds);

    ax.set_xticklabels(labels=[str(int(np.round(1e3*el,0))) for el in xs]);    
    ax.set_yticklabels(labels=[str(int(np.round(1e3*el,0))) for el in ys]);
    
    ax.plot([0,0.045],[0,-(params['params'][0]['m0']+0.045)*(params['params'][0]['D']/params['params'][0]['m1'])],'b-')
    
    ax.patch.set_facecolor('white')
    
    if(params['params'][0]['D'] != 0):
        arr = mpatches.FancyArrowPatch((.02, .019), (.02, .032),
                               arrowstyle='<->,head_width=.15', mutation_scale=20)
        ax.add_patch(arr)
        ax.annotate("$E_u$", (3,0.3), xycoords=arr, ha='center', va='bottom')
        
        arr = mpatches.FancyArrowPatch((.02, -.024), (.02, .013),
                               arrowstyle='<->,head_width=.15', mutation_scale=20)
        ax.add_patch(arr)
        ax.annotate("$E_l$", (3,0.5), xycoords=arr, ha='center', va='bottom')
#    else:
    
    if which_param == 'W':
        ax.set_title("$M_z="+str(params['m_z'][0])+"~eV,~m_0="+str(params['params'][0]['m0'])+"~eV,~m_1="+str(params['params'][0]["m1"])+"~eV\AA^2,$\n $\Delta="+str(params["delta"][0])+"~eV~,D="+str(params['params'][0]["D"])+"~eV\AA^2~,u_B="+str(params['params'][0]['u_B'])+"~eV$", fontsize=fontsize);        
        plt.savefig("phase/phase_simple_Mz"+str(params['m_z'][0])+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".png" ) 
    else:
        ax.set_title("$L_y="+str(syst_W)+"~\AA,~m_0="+str(params['params'][0]["m0"])+"~eV,~m_1="+str(params['params'][0]["m1"])+"~eV\AA^2,$\n $\Delta="+str(params["delta"][0])+"~eV~,D="+str(params['params'][0]["D"])+"~eV\AA^2~,u_B="+str(params['params'][0]['u_B'])+"~eV$", fontsize=fontsize);
        plt.savefig("phase/phase_simple_Ly"+str(syst_W)+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".png" )     

    if not replot:
        if which_param == 'W':
            np.save( arr=[z_topological, z_not_topological], file="phase/phase_Mz"+str(params['m_z'][0])+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".npy" )        
        else:
            np.save( arr=[z_topological, z_not_topological], file="phase/phase_Ly"+str(syst_W)+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".npy" )        


In [ ]:
import matplotlib.patches as mpatches
def create_topological_double(linv1,lgap1,linv2,lgap2,params,Max):
    
    learner_inv=[linv1,linv2]
    learner_gap=[lgap1,lgap2]
                 
    fig, ax = plt.subplots(1,2,figsize=(3.5,1.75),layout='constrained',sharey=True)
    fig.get_layout_engine().set(w_pad=0.01, h_pad=0.0, wspace=0.0, hspace=0.0)
    
    for i in [0,1]:
        gap_interploated=learner_gap[i].interpolated_on_grid(n=1000)
        inv_interploated=learner_inv[i].interpolated_on_grid(n=1000)

        top = inv_interploated[2] > 0
        not_top = inv_interploated[2] < 0
        x, y, z = gap_interploated[0], gap_interploated[1], gap_interploated[2]

        z_topological = z.copy()
        z_topological[top] = -1

        z_not_topological = z.copy()
        z_not_topological[not_top] = -1 

        im1 = ax[i].contourf(x,y,z_topological.T,vmin=0,vmax=Max,levels=np.append(np.linspace(0,Max,50),1),cmap="Reds")#,params_Chen,0.0025)    
        im3 = ax[i].contourf(x,y,z_not_topological.T,vmin=0,vmax=Max,levels=np.append(np.linspace(0,Max,50),1),cmap="gray_r")#,params_Chen,0.0025)

        ax[i].set_xlabel("$M_z$ [meV]");
        xs = np.linspace(p_bounds[0],.01*(p_bounds[1]//.01),int((p_bounds[1]-p_bounds[0])/.01)+1)
        ys = np.linspace(-.01-(mu_bounds[1]-mu_bounds[0])/2,.03+(mu_bounds[1]-mu_bounds[0])/2,+int((.04+mu_bounds[1]-mu_bounds[0])/.02)+1)
        
        ax[i].set_xticks(xs);
        ax[i].set_yticks(ys);
        
        ax[i].set_ylim(min(y),max(y));
        
        ax[i].set_xticklabels(labels=[str(int(np.round(1e3*el,0))) for el in xs]);
        ax[i].set_yticklabels(labels=[str(int(np.round(1e3*el,0))) for el in ys]);
        
        ax[i].patch.set_facecolor('white')
            
    ax[0].set_ylabel("$\mu$ [meV]");
    
    if(params['params'][0]['D'] != 0):
        arr = mpatches.FancyArrowPatch((.02, .019), (.02, .032),
                               arrowstyle='<->,head_width=.15', mutation_scale=20)
        #ax[1].add_patch(arr)
        #ax[1].annotate("$E_u$", (3,0.3), xycoords=arr, ha='center', va='bottom',fontsize=fontsize)

        arr = mpatches.FancyArrowPatch((.02, -.024), (.02, .013),
                               arrowstyle='<->,head_width=.15', mutation_scale=20)
        #ax[1].add_patch(arr)
        #ax[1].annotate("$E_l$", (3,0.5), xycoords=arr, ha='center', va='bottom',fontsize=fontsize)
        
        ax[1].plot([0,0.045],[0,-(params['params'][0]['m0']+0.045)*(params['params'][0]['D']/params['params'][0]['m1'])],'g--',lw=1)
        ax[1].plot([0.04,0.04],[-1,1],'k:',lw=1)

    cbar = fig.colorbar(im1, ax=ax[1],ticks=np.linspace(0,Max,5))
    cbar2 = fig.colorbar(im3, ax=ax[1],ticks=np.linspace(0,Max,5))
    Max_Delta = np.round(Max/params["delta"][0],1)
    cbar2.set_ticklabels( ["" for n in np.linspace(0,Max_Delta,5)])
    cbar.set_ticklabels( [str(np.round(n,2))+"$\\Delta$" for n in np.linspace(0,Max_Delta,5)])
    
    

    ax[1].text(
            0.75, 0.45, 
            '(c)',
            transform=ax[1].transAxes)

    caption = ['(a)','(b)']
    for i in range(len(caption)):
        if i == 0:
            sshift = -0.325
        else:
            sshift = -0.15
        ax[i].text(
            sshift, .985, 
            caption[i],
            transform=ax[i].transAxes)
    
    if which_param == 'W':
        plt.savefig("phase/phase_double_Mz"+str(params['m_z'][0])+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".pdf",bbox_inches='tight')
    else:
        plt.savefig("phase/phase_double_Ly"+str(syst_W)+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".png")
        plt.savefig("phase/phase_double_Ly"+str(syst_W)+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".pdf",bbox_inches='tight')

## Top map

In [ ]:
def create_topological_map(learner_inv,learner_gap,learner_crossings,params,Max,replot):
    
    if not replot:    
        gap_interploated=learner_gap.interpolated_on_grid(n=1000)
        inv_interploated=learner_inv.interpolated_on_grid(n=1000)
        cross_interploated=learner_crossings.interpolated_on_grid(n=1000)

        topological_one = (inv_interploated[2] > 0) | (np.round(cross_interploated[2]) != 1)
        topological_odd = (inv_interploated[2] > 0)  | (np.round(cross_interploated[2])%2 == 0)  | (np.round(cross_interploated[2]) == 1)
        not_topological_even = (inv_interploated[2] < 0)  | (np.round(cross_interploated[2]) == 0)#| (np.round(cross_interploated[2])%2 == 1) | (np.round(cross_interploated[2]) == 0)
        
        x, y, z = gap_interploated[0],gap_interploated[1],gap_interploated[2]
        
        z_topological_one = z.copy()
        z_topological_one[topological_one] = -1
        
        z_topological_odd = z.copy()
        z_topological_odd[topological_odd] = -1

        z_not_topological_even = z.copy()
        z_not_topological_even[not_topological_even] = -1
    else:
        x = np.linspace(*p_bounds,1000)        
        y = np.linspace(*mu_bounds,1000)
        
        z_topological_one = z_topological_one_replot
        z_topological_odd = z_topological_odd_replot
        z_not_topological_even = z_not_topological_even_replot   

    fig, ax = plt.subplots(2,1,figsize=(6,6),layout='constrained', height_ratios=[4,1])
    fontsize=12
    
    if which_param == 'W':
        ax[1].set_xlabel("$W$ [nm]",fontsize = fontsize);
        x = x/params['a'][0]
    else:
        ax[1].set_xlabel("$M_z$ [meV]",fontsize = fontsize);
        x = x*1e3
    

    im2 = ax[0].contourf(x,y,z_topological_odd.T,vmin=0,vmax=Max,levels=np.append(np.linspace(0,Max,50),1),cmap="Blues")#,params_Chen,0.0025)    
    im1 = ax[0].contourf(x,y,z_topological_one.T,vmin=0,vmax=Max,levels=np.append(np.linspace(0,Max,50),1),cmap="Reds")#,params_Chen,0.0025)    
    im3 = ax[0].contourf(x,y,z_not_topological_even.T,vmin=0,vmax=Max,levels=np.append(np.linspace(0,Max,50),1),cmap="gray_r")#,params_Chen,0.0025)

    cbar = fig.colorbar(im1, ax=ax[0],ticks=np.linspace(0,Max,5))
    Max_Delta = np.round(Max/params["delta"][0],1)
    cbar.set_ticklabels( [str(np.round(n,2))+"$\\Delta$" for n in np.linspace(0,Max_Delta,5)],fontsize=fontsize)

    ax[0].set_ylabel("$\mu$ [meV]", fontsize=fontsize);
    ys = np.linspace(mu_bounds[0],mu_bounds[1],int((mu_bounds[1]-mu_bounds[0])/.02)+1)
    ax[0].set_yticks(ys);
    ax[0].set_ylim(*mu_bounds);

    ax[0].set_yticklabels(labels=[str(int(np.round(1e3*el,0))) for el in ys], fontsize=fontsize);

    delta_ind_1 = np.zeros(len(x))        
    delta_ind_t = np.zeros(len(x))
    delta_ind_nt = np.zeros(len(x))

    for i in range(len(x)):
        delta_ind_1[i]  = np.average(z_topological_one[i]    [z_topological_one[i]    !=-1])        
        delta_ind_t[i]  = np.average(z_topological_odd[i]    [z_topological_odd[i]    !=-1])
        delta_ind_nt[i] = np.average(z_not_topological_even[i][z_not_topological_even[i]!=-1])
        
    ax[1].plot(x,delta_ind_1 /params["delta"][0] ,"r-",label='Top. $N=1$')
    ax[1].plot(x,delta_ind_t /params["delta"][0] ,"b-",label='Top. $N>1$')
    ax[1].plot(x,delta_ind_nt/params["delta"][0],"k-",label='Triv. $N>0$')

    ax[1].set_xlim(min(x),max(x))
    ax[1].set_ylim(0,max(delta_ind_nt/params["delta"][0])*1.05)
    ax[1].set_ylabel('$\Delta_{ind}~[\Delta]$', fontsize=fontsize);
    ax[1].legend(frameon=True,fontsize=8,loc='upper right');
    
    ax[0].patch.set_facecolor('white')
    
    if which_param == 'W':
        ax[0].set_title("$M_z="+str(params['m_z'][0])+"~eV,~m_0="+str(params['params'][0]['m0'])+"~eV,~m_1="+str(params['params'][0]["m1"])+"~eV\AA^2,$\n $\Delta="+str(params["delta"][0])+"~eV~,D="+str(params['params'][0]["D"])+"~eV\AA^2~,u_B="+str(params['params'][0]['u_B'])+"~eV$", fontsize=fontsize);        
        plt.savefig("phase/phase_Mz"+str(params['m_z'][0])+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".png" )        
    else:
        ax[0].set_title("$L_y="+str(syst_W)+"~\AA,~m_0="+str(params['params'][0]["m0"])+"~eV,~m_1="+str(params['params'][0]["m1"])+"~eV\AA^2,$\n $\Delta="+str(params["delta"][0])+"~eV~,D="+str(params['params'][0]["D"])+"~eV\AA^2~,u_B="+str(params['params'][0]['u_B'])+"~eV$", fontsize=fontsize);
        plt.savefig("phase/phase_Ly"+str(syst_W)+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".png" )     

    if not replot:
        if which_param == 'W':
            np.save( arr=[z_topological_one, z_topological_odd,z_not_topological_even], file="phase/phase_Mz"+str(params['m_z'][0])+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".npy" )        
        else:
            np.save( arr=[z_topological_one, z_topological_odd,z_not_topological_even], file="phase/phase_Ly"+str(syst_W)+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".npy" )        


## Gaps

In [ ]:
def create_gaps_topological_map(learner_inv,learner_gap,learner_crossings,params,Max,replot):
    
    if not replot:    
        gap_interploated=learner_gap.interpolated_on_grid(n=1000)
        cross_interploated=learner_crossings.interpolated_on_grid(n=1000)

        gapped = (np.round(cross_interploated[2]) == 0)
        
        x, y, z = gap_interploated[0],gap_interploated[1],gap_interploated[2]
        
        z[gapped] = -1
        
        x_points, y_points = learner_gap.to_dataframe()['x'], learner_gap.to_dataframe()['y']
        extra_data = learner_gap.to_dataframe()['extra_data']
        gap_2,gap_3 = [],[]
        for i in range(len(extra_data)):
            gaps = extra_data[i]['gaps'][(extra_data[i]['gaps']<params['delta'][0]/2)&(~np.isnan(extra_data[i]['gaps']))]
            if len(gaps) < 2:
                gap_2.append(-1)
            else:
                gap_2.append(np.sort(gaps)[1])
            if len(gaps) < 2:
                gap_3.append(-1)
            else:
                gap_3.append(np.mean(gaps[1:]))               
        points, values = learner_gap._data_in_bounds()
        xx, yy = np.meshgrid(x,y)
        
        ip2 = LinearNDInterpolator(np.vstack([x_points,y_points]).T,gap_2,rescale=True)
        ip3 = LinearNDInterpolator(np.vstack([x_points,y_points]).T,gap_3,rescale=True)
        z2 = ip2(xx,yy).T
        z3 = ip3(xx,yy).T
        
        z2[z2 < 0], z3[z3 < 0] = -1, -1
        z2[(np.round(cross_interploated[2]) < 2)] = -1
        z3[(np.round(cross_interploated[2]) < 2)] = -1
                
    else:
        x = np.linspace(*p_bounds,1000)        
        y = np.linspace(*mu_bounds,1000)
        
        z = z_replot

    fig, ax = plt.subplots(2,3,figsize=(10,5),layout='constrained', height_ratios=[4,1],width_ratios=[1,1,1],sharex='row',sharey='row')
    fontsize=12
    
    if which_param == 'W':
        ax[1,0].set_xlabel("$W$ [nm]",fontsize = fontsize);
        ax[1,1].set_xlabel("$W$ [nm]",fontsize = fontsize);
        ax[1,2].set_xlabel("$W$ [nm]",fontsize = fontsize);
        x = x/params['a'][0]
    else:
        ax[1,0].set_xlabel("$M_z$ [meV]",fontsize = fontsize);
        ax[1,1].set_xlabel("$M_z$ [meV]",fontsize = fontsize);
        ax[1,2].set_xlabel("$M_z$ [meV]",fontsize = fontsize);
        x = x*1e3        
    
    im1 = ax[0,0].contourf(x,y,z.T,vmin=0,vmax=Max,levels=np.append(np.linspace(0,Max,50),1),cmap="Reds")#,params_Chen,0.0025)
    im2 = ax[0,1].contourf(x,y,z2.T,vmin=0,vmax=Max,levels=np.append(np.linspace(0,Max,50),1),cmap="Blues")#,params_Chen,0.0025)
    im3 = ax[0,2].contourf(x,y,z3.T,vmin=0,vmax=Max,levels=np.append(np.linspace(0,Max,50),1),cmap="Greens")#,params_Chen,0.0025)
    cbar = fig.colorbar(im1, ax=ax[0],ticks=np.linspace(0,Max,5))
    Max_Delta = np.round(Max/params["delta"][0],1)
    cbar.set_ticklabels( [str(np.round(n,2))+"$\\Delta$" for n in np.linspace(0,Max_Delta,5)],fontsize=fontsize)

    ax[0,0].set_ylabel("$\mu$ [meV]", fontsize=fontsize);
    ys = np.linspace(mu_bounds[0],mu_bounds[1],int((mu_bounds[1]-mu_bounds[0])/.02)+1)
    ax[0,0].set_yticks(ys);
    ax[0,0].set_ylim(*mu_bounds);
    ax[0,0].set_yticklabels(labels=[str(int(np.round(1e3*el,0))) for el in ys], fontsize=fontsize);

    delta_ind,delta_ind2,delta_ind3 = np.zeros(len(x)), np.zeros(len(x)), np.zeros(len(x))        
    
    for i in range(len(x)):
        delta_ind[i]  = np.average(z[i] [z[i] !=-1])        
        delta_ind2[i] = np.average(z2[i] [z2[i] !=-1])        
        delta_ind3[i] = np.average(z3[i] [z3[i] !=-1]) 
        
    ax[0,0].set_title("Smallest gap")
    ax[0,1].set_title("2nd smallest gap")
    ax[0,2].set_title("Avg. gap (w/out smallest)")
    
    ax[1,0].plot(x,delta_ind /params["delta"][0] ,"r-")
    ax[1,1].plot(x,delta_ind2 /params["delta"][0] ,"b-")
    ax[1,2].plot(x,delta_ind3 /params["delta"][0] ,"g-")

    ax[1,0].set_xlim(min(x),max(x))
    ax[1,0].set_ylim(0,max([max(delta_ind),max(delta_ind2),max(delta_ind3)])/params["delta"][0]*1.05)
    ax[1,0].set_ylabel('$\Delta_{ind}~[\Delta]$', fontsize=fontsize);

#    ax[1,0].legend(frameon=True,fontsize=7,loc='upper right');
    
    ax[0,0].patch.set_facecolor('white')
    
    if which_param == 'W':
        fig.suptitle("$M_z="+str(params['m_z'][0])+"~eV,~m_0="+str(params['params'][0]['m0'])+"~eV,~m_1="+str(params['params'][0]["m1"])+"~eV\AA^2,\Delta="+str(params["delta"][0])+"~eV~,D="+str(params['params'][0]["D"])+"~eV\AA^2~,u_B="+str(params['params'][0]['u_B'])+"~eV$", fontsize=fontsize);        
        plt.savefig("phase/phase_gaps_Mz"+str(params['m_z'][0])+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".png" )        
    else:
        fig.suptitle("$L_y="+str(syst_W)+"~\AA,~m_0="+str(params['params'][0]["m0"])+"~eV,~m_1="+str(params['params'][0]["m1"])+"~eV\AA^2,\Delta="+str(params["delta"][0])+"~eV~,D="+str(params['params'][0]["D"])+"~eV\AA^2~,u_B="+str(params['params'][0]['u_B'])+"~eV$", fontsize=fontsize);
        plt.savefig("phase/phase_gaps_Ly"+str(syst_W)+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".png" )

    if not replot:
        if which_param == 'W':
            np.save( arr=[z], file="phase/phase_gaps_Mz"+str(params['m_z'][0])+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".npy" )        
        else:
            np.save( arr=[z], file="phase/phase_gaps_Ly"+str(syst_W)+"_m0"+str(params['params'][0]["m0"])+"_m1"+str(params['params'][0]["m1"])+"_Delta"+str(params["delta"][0])+"_D"+str(params['params'][0]["D"])+"_u_B"+str(params['params'][0]['u_B'])+"_minp"+str(p_bounds[0])+"_maxp"+str(p_bounds[1])+".npy" )        

## Plotting

In [ ]:
#qa_top.learner.learners[0].interpolated_on_grid()

In [ ]:
#qa_top.learner.learners[0].to_dataframe()['extra_data'][0]['gaps']

In [ ]:
linv=qa_mn.learner.learners[0]
lgap=qa_top.learner.learners[0]

In [ ]:
create_topological_simple(qa_mn.learner.learners[0],qa_top.learner.learners[0],params_2D,0.001*.4,False)

In [ ]:
create_topological_double(linv,lgap,qa_mn.learner.learners[0],qa_top.learner.learners[0],params_2D,0.001*.4)

In [ ]:
create_gaps_topological_map(qa_mn.learner.learners[0],qa_top.learner.learners[0],qa_ph.learner.learners[0],params_2D,0.001*.4,False)

In [ ]:
thres1 = 50
thres2 = 0.001
data = qa_top.learner.learners[0].to_dataframe()
for i in range(len(data)):
    if( abs(data['x'][i] - 1000) < thres1 and abs(data['y'][i] - 0) < thres2):
        print(i,data['extra_data'][i]['gap']/params_2D['delta'][0])

In [ ]:
ii = 1132

In [ ]:
data['extra_data'][ii]['gap']#.keys()

In [ ]:
data['x'][ii], data['y'][ii]

In [ ]:
print(data['extra_data'][ii]['momenta_cros'])
print(data['extra_data'][ii]['momenta_gap'])
print(data['extra_data'][ii]['gaps'])
print(data['extra_data'][ii]['gap']/0.001)

In [ ]:
executor = client.executor()

In [ ]:
job = executor.submit(funcs.gap_search_k, a=10, a_z=10, T=1, W=data['x'][ii], mu=data['y'][ii], m_z=2*M0, delta=0.001, h_mode=1, params=params_Chen)

In [ ]:
res = job.result()

In [ ]:
E_k0, wfs_k0, rhos = res

In [ ]:
plt.plot(np.ones(len(E_k0)), E_k0, "x")
plt.ylim(-0.001,0.001)

In [ ]:
E_k0 = E_k0.real
#wfs_k0 = wfs_k0[:, E_k0 > 0]
#E_k0 = E_k0[E_k0 > 0]
rhos_p_k0 = np.array([rhos['p'](wf) for wf in wfs_k0.T])
rhos_h_k0 = np.array([rhos['h'](wf) for wf in wfs_k0.T])
densitys_ph = rhos_p_k0 - rhos_h_k0
fist_h_band_index = np.argmax(densitys_ph > 0)
fist_h_band_E = E_k0[fist_h_band_index]

In [ ]:
plt.plot(densitys_ph,E_k0,"x")
#plt.ylim(-0.003,0.003)
plt.plot([-1,1],[0,0],"k--")
plt.ylabel("E [eV]")
plt.xlabel("densitys_ph")

In [ ]:
print(res['momenta_cros'])
print(res['momenta_gap'])
print(res['gaps'])
print(res['gap'])

In [ ]:
import systems
import kwant

In [ ]:
W = 20
T = 30
a = 10
lead2 = systems.make_lead(
        a, 1, W, 1, L=0, ph_symmetry = False,
        conservation_law=None, directions='x',ham_type='2D')

In [ ]:
lead_f = lead.finalized()

In [ ]:
Ham = lead_f.hamiltonian_submatrix(params=dict(hbar=1,v_F=3,D=0,delta=0,m0=1,m1=1,m_z=0,mu_ti=0))

In [ ]:
len(Ham[0])

In [ ]:
lead_f.id_by_site

In [ ]:
#lead_f.inter_cell_hopping(params=dict(hbar=1,v_F=3,D=0,delta=0,m0=1,m1=1,m_z=0,mu_ti=0))

In [ ]:
lead_f.hamiltonian(1,0,params=dict(hbar=1,v_F=3,D=1,delta=0,m0=0.3,m1=1,m_z=0,mu_ti=0))

In [ ]:
lead_f.hamiltonian(0,1,params=dict(hbar=1,v_F=3,D=0,delta=0,m0=2,m1=2,m_z=0,mu_ti=0))

In [ ]:
lead

In [ ]:
reload()

In [ ]:
W = 12
T = 12
a = 4
lead2 = systems.make_lead(
        a, 1, W, T, L=0, ph_symmetry = False,
        conservation_law=None, directions='x',ham_type='2D_sides')

In [ ]:
def get_where_side():
    def where_side(site,a,W,T):
        left_edge = (T-a)/2
        right_edge = W + (T-a)/2
        return left_edge <= site.pos[1] <= right_edge
    return where_side

In [ ]:
lead2_f = lead2.finalized()

In [ ]:
#kwant.plotter.set_engine("plotly")
kwant.plotter.set_engine("matplotlib")
import matplotlib.pyplot as plt, matplotlib.backends
plot = kwant.plot(lead2, show=False)
plot.show()

In [ ]:
list(lead2.sites())

In [ ]:
lead2.H#[list(lead2.sites())[0],list(lead2.sites())[1]]

In [ ]:
lead2[list(lead2.sites())[1]]

In [ ]:
pparams = dict(hbar=1,v_F=3,D=1,delta=0,m0=0.3,m1=1,m_z=0,mu_ti=0,where_side=get_where_side(),W=W,T=T,a=a)

In [ ]:
lead2_f.sites

In [ ]:
lead2_f.hamiltonian_submatrix(params=pparams) = 0

In [ ]:
for i in range(6):
    print(lead2_f.hamiltonian(i,i,params=pparams))

In [ ]:
lead2_f.cell_hamiltonian(params=pparams)[1]

In [ ]:
lead2_f.hamiltonian_submatrix(params=pparams)[:4,:4] = np.array([[-0.04+0.j,  0,  0.  +0.j,  0.  +0.j],
       [ 0.26+0.j, -0.04+0.j,  0.  +0.j,  0.  +0.j],
       [ 0.  +0.j,  0.  +0.j, -0.04+0.j,  0.26+0.j],
       [ 0.  +0.j,  0.  +0.j,  0.26+0.j, -0.04+0.j]])

In [ ]:
lead2_f.hamiltonian_submatrix(params=pparams)[:4,:4]

In [ ]:
for i in range(4):
    print(lead_f.hamiltonian(i,i+1,params=pparams))